In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
df_first = pd.read_csv('myntra-train-cat-edited.csv',low_memory=False)

In [ ]:
len(df_first)

In [ ]:
df_first.head()

In [ ]:
# for i,row in df_first.iterrows():
#     print(i)
#     pic_path = 'data/hackerearth-myntra/train/'+row['Link_to_the_image']
#     try:
#         im = open_image(pic_path)
#         df_first.loc[i,'found']=True
#     except:
#         df_first.loc[i,'found']=False
    

In [ ]:
torch.cuda.is_available()

In [ ]:
# df_use.to_csv('myntra-category-usable-images.csv',index=False)

In [ ]:
# len(df_use)

In [ ]:
df = pd.read_csv('myntra-category-usable-images.csv',low_memory = False)

In [ ]:
# df = df[['Link_to_the_image','Sub_category']]

In [ ]:
# df.to_csv('myntra-last.csv',index=False)

In [ ]:
# for i,row in df_first.iterrows():
#     count = 0
#     if '?' in row['Link_to_the_image']:
#         count= count+1
#         df_first.drop(df_first.index[i])
# len(df_first+" "+str(count))

In [ ]:
# df_first.to_csv('myntra-train-21-1.csv',index=False)

In [ ]:
df_last = pd.read_csv('myntra-last.csv',low_memory=False)

In [ ]:
df_last.head()

In [ ]:
sz = 512
arch = dn121
bs = 64
path='data/hackerearth-myntra'

In [ ]:
trn_tfms,val_tfms = tfms_from_model(arch,sz,crop_type=CropType.CENTER)

In [ ]:
data = ImageClassifierData.from_csv(path='data/hackerearth-myntra/',folder='train',csv_fname='myntra-last.csv',tfms=(trn_tfms,val_tfms),bs=bs,num_workers=0)

In [ ]:
# %cd $path
# %rm -r tmp

In [ ]:
learn = ConvLearner.pretrained(arch,data,precompute=True,ps=0.20)

In [ ]:
lrf=learn.lr_find()
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(0.05,2)

In [ ]:
learn.fit(0.01,2)

In [ ]:
learn.fit(0.001,3)

In [ ]:
lr = 0.1
lrs = np.array([lr/9,lr/3,lr])

In [ ]:
learn.unfreeze()
learn.fit(lrs, 2, cycle_len=1, cycle_mult=2)

In [ ]:
learn.fit(lr, 4, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('21-4thExp')

In [ ]:
learn.load('21-3rdExpAugmented')

In [ ]:
log_preds,y = learn.TTA()


In [ ]:
probs = np.mean(np.exp(log_preds),0)

In [ ]:
accuracy_np(probs, y)

In [ ]:
y[10]=9

In [ ]:
predictions = np.argmax(probs,axis=1)

In [ ]:
trn_tfms,val_tfms=tfms = tfms_from_model(arch, sz,crop_type=CropType.RANDOM, max_zoom=1.1)
data = ImageClassifierData.from_csv(path='data/hackerearth-myntra/',folder='train',csv_fname='myntra-last.csv',tfms=(trn_tfms,val_tfms),bs=bs,num_workers=0)

In [ ]:
learn.fit(0.001,5)

In [ ]:
# learn.save('21-3rdExpAugmented')

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, predictions)

In [ ]:
def plot_confusion_matrix_mine(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.figure(figsize=(12,12))

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
plot_confusion_matrix_mine(cm,data.classes,normalize=False)

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask((preds == data.val_y)==is_correct & (data.val_y == y), mult)

In [ ]:
preds = predictions
PATH = 'data/hackerearth-myntra/'

In [ ]:
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [ ]:
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit(0.01,1)

In [ ]:
learn.summary()

In [ ]:
fname = '/home/paperspace/data/hackerearth-myntra/test/11476446167866-2GO-Women-Grey-Solid-V-Neck-T-Shirt-3201476446167715-1.jpg'

In [ ]:
learn.freeze()

In [ ]:
test = open_image(fname)
test= tfms[1]((test))
test.shape
# print(test.shape)
# cv2.imwrite('test.png',test)



In [ ]:
pred = learn.predict_array(test[None])
data.classes[np.argmax(pred[0,:])]

In [ ]:
import pandas as pd

path = 'data/hackerearth-myntra'

In [ ]:
df = pd.read_csv(f'myntra-submission_online-final.csv', low_memory=False)

In [ ]:
df.head()

In [ ]:
count = 0
for i,row in df.iterrows():
    try:
        pic_link = row['Link_to_the_image']
        if "?dl=1" in pic_link:
            count= count+1
            df.loc[i,'Link_to_the_image']=pic_link.replace("?dl=1","")
    except:
        print("noFound")

In [ ]:
count

In [ ]:
# df.to_csv('myntra-submission_online-final.csv',index=False)

In [ ]:
count =0
for i,row in df.iterrows():
    pic_link = row['Link_to_the_image'] 
    try:
        pic_name_temp = pic_link.split('/')
        pic_name = pic_name_temp[len(pic_name_temp)-1]
        full_path_name = '/home/paperspace/fastai/courses/dl1/'+path+'/test/'+pic_name
        img = tfms[1](open_image(full_path_name))
        pred = learn.predict_array(img[None])
        temp_data = data.classes[np.argmax(pred)]
        df.loc[i,'Sub_category']=temp_data
        print(str(i)+temp_data)
    except:
        print(pic_link)
        df.loc[i,'Sub_category']='Solid'
print(count)

In [ ]:
df.tail()

In [ ]:
df.to_csv('22-myntra-1.csv',index=False)